#### Find one cell that its neg_low is above vmzoro and one cell that its neg_low is above 0; look at the trace data for both on dvt.
Use data set: 
1. 170109_ENG-SYS_02_wobbuffet_WAV13R07C12_cycle1 (check is existed: /experiments/vmzero_end/cell_anno/vmzero_end Dataset {131072) 
2. 170124_SIG-A_03_blastoise_WAV06R05C15  (check is existed: /experiments/vmzero_end/cell_anno/vmzero_end Dataset {131072) 
3. 170206_TAG_01_caterpie_WAV07R10C15 (CVC mode ==> vmzero_end is NOT existed, why?)

In [2]:
from os.path import abspath, dirname, join
import numpy as np
import matplotlib.pyplot as plt

import ac_analysis

from ac_analysis.model.annotations import load_from_h5

### 1) Load the annotation files and show the interested annotations

In [3]:
# 1) 170109_ENG-SYS_02_wobbuffet_WAV13R07C12_cycle1
#anno_path="/mnt/AGZ_Home_vmwin10/workspace_pOD/genia/Andrew/170109_ENG-SYS_02_wobbuffet_WAV13R07C12_cycle1/P_00_170110023618_ggc3-keeper_ac-analysis_v11.19.1/annotations.h5"
anno_path="/home/genia/rigdata/wobbuffet/170109_ENG-SYS_02_wobbuffet_WAV13R07C12_cycle1/\
P_01_170212055345_spore1-master_ac-analysis_v11.22.1/annotations.h5"
# 2) 170124_SIG-A_03_blastoise_WAV06R05C15
#anno_path="/mnt/AGZ_Home_vmwin10/workspace_pOD/genia/Andrew/170124_SIG-A_03_blastoise_WAV06R05C15/annotations.h5"

annos = load_from_h5(anno_path)
print "annos ==> ", annos

annos ==>  <ac_analysis.model.annotations_v2.Annotations_v2 object at 0x7f60c51034d0>


In [7]:
neg_low, units = annos.get_rep_annotation('neg_low') #grab neg_low matrix shape = (cells, reps)
print "neg_low ==> ",  neg_low, units
print "\nneg_low.shape ==> ", neg_low.shape
print "\nneg_low.size ==> ", neg_low.size # ??? Sometime it will show the different number: 4063232, why?

neg_low ==>  [[         nan]
 [         nan]
 [-20.97039986]
 ..., 
 [         nan]
 [         nan]
 [         nan]] ADC

neg_low.shape ==>  (131072, 1)

neg_low.size ==>  131072


In [6]:
cells = annos.cells  #grab all cells
print "cells ==> ", cells
print "\ncells.shape ==> ", cells.shape
print "\ncells.size ==> ", cells.size 

cells ==>  ['b00c0000' 'b00c0001' 'b00c0002' ..., 'b15c8189' 'b15c8190' 'b15c8191']

cells.shape ==>  (131072,)

cells.size ==>  131072


### 2. Filter for neg_low

In [8]:
# Remove rows with only nan and zero 
#(probably dont want to just remove zeros, why?)
mask = np.all(np.isnan(neg_low) | np.equal(neg_low, 0), axis=1) 
print "mask is ==> ", mask

filt_neg_low = neg_low[~mask]   #only get cells where neg_low is not all nan or all zero
print "\nfilt_neg_low is: ", filt_neg_low

filt_cells = cells[~mask]  #get cell names based on neg_low mask 
print "\nfilt_cells is: ", filt_cells

# Test one cell trace:
neg_low_cell = neg_low[cells == 'b15c5003']
print "\nneg_low_cell is: ", neg_low_cell

mask is ==>  [ True  True False ...,  True  True  True]

filt_neg_low is:  [[-20.97039986]
 [-50.69053268]
 [-65.80702209]
 ..., 
 [-61.30174637]
 [-62.99001312]
 [-62.3810997 ]]

filt_cells is:  ['b00c0002' 'b00c0003' 'b00c0011' ..., 'b15c7839' 'b15c7841' 'b15c7850']

neg_low_cell is:  [[ nan]]


### 3. Get oc_calibration_vmzero

In [9]:
# 1) Read oc_calibration_vmzero
vmzeros, unit = annos.get_cell_annotation('oc_calibration_vmzero')
print "1) vmzeros ==> ", vmzeros, unit

# 2) returns numpy matrix, and string (matrix, string)
# One of the run that does not have the nvmzeros_end (CVC mode one, why? )
vmzeros_end, unit = annos.get_cell_annotation('vmzero_end') 
print "\n2) vmzeros_end, unit ==> ", vmzeros_end, unit

# 3) Filted vmzero that elimited the nan, etc??
filt_vmzeros = vmzeros[~mask]
#plt.hist(filt_vmzeros, bins=100); plt.show()  #plot to show vmzeoro distribution
print "\n3) filt_vmzeros ==> ",  filt_vmzeros 

# 4) what you have is neg_low minus vmzero:
#    Get flatten matrix (numpy.matrix.flatten), 
#    See: https://docs.scipy.org/doc/numpy/reference/generated/numpy.matrix.flatten.html
#       https://plot.ly/numpy/ravel/

# now what you have is neg_low minus vmzero
flat_filt_neg_low = np.ravel(filt_neg_low)  #flatten matrix to array to do subtraction of vmzero 
print "4-1) flat_filt_neg_low ==> ", flat_filt_neg_low

#subtract vmzero:
neg_low_minus_vmzero = flat_filt_neg_low - filt_vmzeros  #subtract vmzero
print "4-2) neg_low_minus_vmzero ==> ", neg_low_minus_vmzero

#confirm proper shape
print "4-3) neg_low_minus_vmzero.shape ==> ", neg_low_minus_vmzero.shape  #confirm proper shape
print "4-4) flat_filt_neg_low.shape ==> ",  flat_filt_neg_low.shape

# Finally, give a value, such as : 0, < 4, < 5:
# How to decide?

final_mask = [neg_low_minus_vmzero < 5]
print "\n5-1) final_mask ==> ", final_mask

print "5-2) filt_cells[final_mask] ==> ", filt_cells[final_mask]
print "5-3) The size of the filt_cells[final_mask] ==> ",  filt_cells[final_mask].size
# 8700

# A test: meaning of the following samples:
print "\n6) Check a few cells... ==> ", filt_cells[final_mask][1:30]

1) vmzeros ==>  [          nan           nan           nan ...,  111.48249054  117.55555725
           nan] ADC

2) vmzeros_end, unit ==>  [ 111.53581238  131.98692322  126.56000519 ...,  117.90969849  125.96927643
  107.09092712] ADC

3) filt_vmzeros ==>  [          nan           nan  111.98482513 ...,  109.4708252   125.0655899
  116.39612579]
4-1) flat_filt_neg_low ==>  [-20.97039986 -50.69053268 -65.80702209 ..., -61.30174637 -62.99001312
 -62.3810997 ]
4-2) neg_low_minus_vmzero ==>  [          nan           nan -177.7918396  ..., -170.77256775 -188.05560303
 -178.77722168]
4-3) neg_low_minus_vmzero.shape ==>  (8700,)
4-4) flat_filt_neg_low.shape ==>  (8700,)

5-1) final_mask ==>  [array([False, False,  True, ...,  True,  True,  True], dtype=bool)]
5-2) filt_cells[final_mask] ==>  ['b00c0011' 'b00c0031' 'b00c0032' ..., 'b15c7839' 'b15c7841' 'b15c7850']
5-3) The size of the filt_cells[final_mask] ==>  8031

6) Check a few cells... ==>  ['b00c0031' 'b00c0032' 'b00c0033' 'b00c0034' 'b

### 4. Finally, calculate the difference (range from 0 ~ 5?) 

In [10]:
vmzero_minus_neg_low = filt_vmzeros - flat_filt_neg_low
print "1) vmzero_minus_neg_low ==> ", vmzero_minus_neg_low

final_mask = [vmzero_minus_neg_low > 0]
print "\n2) final_mask ==> ", final_mask

print "\n3) vmzero_minus_neg_low.max() ==> ", vmzero_minus_neg_low.max()
print "\n3-1) vmzero_minus_neg_low.min() ==> ", vmzero_minus_neg_low.min()

cell_vmzero = filt_vmzeros[filt_cells == 'b00c0486']  # What does this mean?

print "4-1) cell_vmzero at cell_ID of ==> ", cell_vmzero

1) vmzero_minus_neg_low ==>  [          nan           nan  177.7918396  ...,  170.77256775  188.05560303
  178.77722168]

2) final_mask ==>  [array([False, False,  True, ...,  True,  True,  True], dtype=bool)]

3) vmzero_minus_neg_low.max() ==>  nan

3-1) vmzero_minus_neg_low.min() ==>  nan
4-1) cell_vmzero at cell_ID of ==>  [ nan]
